# discover_symmetry.py test notebook

### Todos

- (a) Improve the clarity/user-friendliness of the isclose-type check

- (b) Incorporate the cardinality checks!

- (c) Incorporate the inflation level checks!

NB: for (b-c), change the internal representation of NetworkGraph to be based on the networkx library. Can implement checks of cardinalities & inflation levels by adequate "coloring" (additional data use for isomorphism purposes).

### Logic behind the Distr.is_symmetry() code
Given some agent_perm $\sigma_p$ and io_perm $\pi_{io}$, as well as some distribution $p$, we want to test whether or not, for all $(a_1,\dots,a_n)$ (these should be understood as a set of outputs - forget about inputs, they do not really change the comprehension), it holds that $\pi_{io} \circ \sigma_p(p)[a_1,\dots,a_n] = p[a_1,\dots,a_n]$ $(\star)$. This order reflects our convention to apply, at the level of $p$, first the agent permutation $\sigma_p$ and then the io permutation $\pi_{io}$. To give meaning to this equation, we first have to specify what the different symmetry actions really mean.

##### Representations of io_perm

We define: $\pi_{io}(a_1,\dots,a_n) = (\pi_{io}^{(1)}(a_1),\dots,\pi_{io}^{(n)}(a_n))$, i.e., $\pi_{io}$ consists of $n$ individual permutations of outcomes. This is clearly a valid representation (preserves group structure).

Now, at the level of the distribution $p$, we define: $\pi_{io}(p)[\vec a] = p[\pi_{io}^{-1}(\vec a)]$. This is a valid representation because
$\tau(\pi(p)[\vec a]) = \pi(p)[\tau^{-1}(\vec a)] = p[\pi^{-1} \circ \tau^{-1}(\vec a)] = (\tau\circ\pi)(p)[\vec a]$.

##### Representations of agent_perm

We define: $\sigma_p(a_1,\dots,a_n) = (a_{\sigma_p^{-1}(1)}, \dots, a_{\sigma_p^{-1}(n)})$. 
This is a valid representation: indeed, $\tau(\sigma(a_1,\dots,a_n)) = \tau(a_{\sigma^{-1}(1)}, \dots, a_{\sigma^{-1}(n)}) = (a_{\sigma^{-1}\circ \tau^{-1}(1)},\dots, a_{\sigma^{-1}\circ \tau^{-1}}(n)) = (\tau \circ \sigma)(a_1,\dots,a_n)$.

At the level of the distribution $p$, we define: $\sigma_p(p)[\vec a] = p[\sigma_p^{-1}(\vec a)]$. 
This is a valid representation for the same reasons as $\pi_{io}$ above.

##### Simplifying the symmetry condition

Let's look at the $(\star)$ equation. It really means that we should have $p[\sigma_p^{-1} \circ \pi_{io}^{-1}(\vec a)] = p[\vec a]$ for all $\vec a$, or, by the bijectivity of the group representations, letting $\vec a \mapsto \pi_{io} \circ \sigma_p(\vec a)$, that we should have $p[\vec a] = p[\pi_{io} \circ \sigma_p(\vec a)]$.

In other words, we want, for all $\vec a$, given $\vec b = \sigma_p(\vec a)$ and $\vec c = \pi_{io}(\vec b)$, that $p[\vec c] = p[\vec a]$. The code does exactly that: first, it computes $\vec b$, which is given by

$\vec b = \sigma_p(\vec a) = (a_{\sigma_p^{-1}(1)}, \dots, a_{\sigma_p^{-1}(n)})$, or in other words, $b_i = a_{\sigma_p^{-1}(i)}$, or better put, $b_{\sigma_p(i)} = a_i$.

Then it computes $\vec c = \pi_{io}(\vec b)$, or in other words, $c_i = \pi_{io}^{(i)}(b_i)$.

### Some tests

In [1]:
from discoversym import *
from test_discoversym import *

In [2]:
g = get_triangle_network()
g

----- GRAPH -----
Source alpha is connected to agents A, B
Source  beta is connected to agents B, C
Source gamma is connected to agents A, C
-----------------

In [3]:
p = Distr([2,2,2,1,1,1])
p[0,0,1,0,0,0] = 0.5
p[1,1,0,0,0,0] = 0.5

In [4]:
print(p)
symmetries = p.get_symmetries(g)
for i,sym in enumerate(symmetries):
    if not ( (i == 1) or (i == 2) or (i == 6) ):
        # To limit the size of the output
        continue
    if True:
        if i==0:
            print("[g]                       Identity ")
        if i==1:
            print("[g]                   g_{all swap} =                                Swap 0 & 1 for all")
        if i==2:
            print("[g]                         g_{BC} = Swap Bob and Charlie,          then 0 & 1 for Bob and Charlie")
        if i==3:
            print("[g]            g_{all swap}.g_{BC} = Swap Bob and Charlie,          then 0 & 1 for Alice")
        if i==4:
            print("[g]                         g_{AB} = Swap Alice and Bob")
        if i==5:
            print("[g]            g_{all swap}.g_{AB} = Swap Alice and Bob,            then 0 & 1 for all")
        if i==6:
            # First cyclic:
            # A = 1, B = 0, C = 0   =   0.5
            # A = 0, B = 1, C = 1   =   0.5
            # Then swap 0 & 1 for new Bob
            # A = 1, B = 1, C = 0   =   0.5
            # A = 0, B = 0, C = 1   =   0.5
            print("[g]     g_{all swap}.g_{AB}.g_{BC} = Cycle Alice -> Bob -> Charlie, then 0 & 1 for new Bob")
        if i==7:
            print("[g]                  g_{AB}.g_{BC} = Cycle Alice -> Bob -> Charlie, then 0 & 1 for Alice and Charlie")
        if i==8:
            print("[g]              (g_{AB}.g_{BC})^2 = Cycle Alice -> Charlie -> Bob, then 0 & 1 for Bob and Charlie")
        if i==9:
            print("[g] g_{all swap}.(g_{AB}.g_{BC})^2 = Cycle Alice -> Charlie -> Bob, then 0 & 1 for Alice")
        if i==10:
            print("[g]                         g_{AC} = Swap Alice and Charlie,        then 0 & 1 for Bob")

        # Overall, we see that we obtain a group of twelve elements essentially isomorphic to S_2 \times S_3 (i.e., can freely invert all bits (S_2), as well as freely exchange all the agents provided this is compensated by appropriate outcome symmetries (S_3)).

    print(sym.to_ugly_string())
    print(sym)
    print("")

----- DISTR -----
A = 0, B = 0, C = 1 | X = 0, Y = 0, Z = 0 = 0.5
A = 1, B = 1, C = 0 | X = 0, Y = 0, Z = 0 = 0.5
-----------------
[g]                   g_{all swap} =                                Swap 0 & 1 for all
[Sym][Ugly] agents A->A, B->B, C->C then io A: (0->1, 1->0), B: (0->1, 1->0), C: (0->1, 1->0), X: (0->0), Y: (0->0), Z: (0->0) sources alpha->alpha,  beta-> beta, gamma->gamma
[Sym] Events: A(0|0) A(1|0) B(0|0) B(1|0) C(0|0) C(1|0)
           to A(1|0) A(0|0) B(1|0) B(0|0) C(1|0) C(0|0)
     Sources: []
           to []

[g]                         g_{BC} = Swap Bob and Charlie,          then 0 & 1 for Bob and Charlie
[Sym][Ugly] agents A->A, B->C, C->B then io A: (0->0, 1->1), B: (0->1, 1->0), C: (0->1, 1->0), X: (0->0), Y: (0->0), Z: (0->0) sources alpha->gamma,  beta-> beta, gamma->alpha
[Sym] Events: B(0|0) B(1|0) C(0|0) C(1|0)
           to C(1|0) C(0|0) B(1|0) B(0|0)
     Sources: ['S0', 'S2']
           to ['S2', 'S0']

[g]     g_{all swap}.g_{AB}.g_{BC} = Cycle A